In [1]:
import pandas as pd
import afqinsight.nn.tf_models as nn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from afqinsight.datasets import AFQDataset
from afqinsight.nn.tf_models import cnn_lenet, mlp4, cnn_vgg, lstm1v0, lstm1, lstm2, blstm1, blstm2, lstm_fcn, cnn_resnet
from sklearn.impute import SimpleImputer
import os.path
# Harmonization
from sklearn.model_selection import train_test_split
from neurocombat_sklearn import CombatModel
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error

import pickle
from tools import load_data, augment_this, model_fit

/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
X, y, site = load_data()

In [3]:
from sklearn.utils import shuffle, resample

In [4]:
from sklearn.metrics import r2_score, median_absolute_error

In [5]:
model_dict = {"cnn_lenet": {"model": cnn_lenet, "lr": 0.001}, 
              # "mlp4": {"model": mlp4, "lr": 0.001},
              # "cnn_vgg": {"model": cnn_vgg, "lr": 0.001},
              # "lstm1v0": {"model": lstm1v0, "lr": 0.01},
              # "lstm1": {"model": lstm1, "lr": 0.01},
              # "lstm2": {"model": lstm2, "lr": 0.01},
              # "blstm1": {"model": blstm1, "lr": 0.01},
              # "blstm2": {"model": blstm1, "lr": 0.01},
              # "lstm_fcn": {"model": lstm_fcn, "lr": 0.01},
              # "cnn_resnet": {"model": cnn_resnet, "lr": 0.01}
             }

### cnn_lenet

In [6]:
n_runs = 10

In [7]:
seeds = np.array([np.abs(np.floor(np.random.randn()*1000)) for ii in range(n_runs)], dtype=int)

In [8]:
def fit_and_eval(model, X, y, random_state, batch_size=128, n_epochs=1000, augment=None):
    model_func = model_dict[model]["model"]
    lr = model_dict[model]["lr"]
    X_train, X_test, y_train, y_test, site_train, site_test = train_test_split(X, y, site, test_size=0.2, random_state=random_state)
    imputer = SimpleImputer(strategy="median")
    # Impute train and test separately:
    X_train = np.concatenate([imputer.fit_transform(X_train[..., ii])[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([imputer.fit_transform(X_test[..., ii])[:, :, None] for ii in range(X_test.shape[-1])], -1)
    # Combat
    X_train = np.concatenate([CombatModel().fit_transform(X_train[..., ii], site_train[:, None], None, None)[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([CombatModel().fit_transform(X_test[..., ii], site_test[:, None], None, None)[:, :, None] for ii in range(X_test.shape[-1])], -1)
    
    trained = model_fit(model_func, X_train, y_train, lr, 
                        batch_size=batch_size, n_epochs=n_epochs, 
                        augment=augment)
    metric = []
    value = []
    
    y_pred = trained.predict(X_test)
    metric.append("mae")
    value.append(mean_absolute_error(y_test, y_pred))
    metric.append("mad")
    value.append(median_absolute_error(y_test, y_pred))
    metric.append("r2")
    value.append(r2_score(y_test, y_pred))
    
    result = {'Model': [model] * len(metric),
              'Metric': metric,
              'Value': value}
    return pd.DataFrame(result)

In [ ]:
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval = fit_and_eval(
            model, 
            X, 
            y, 
            random_state=seeds[ii],
            augment=augment_this)
        dfs.append(this_eval)
        one_df = pd.concat(dfs)
        one_df.to_csv("all_sites_online_aug.csv")

##################################################
model:  cnn_lenet
run:  0
pooling layers: 4
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 72)]         0         
                                                                 
 conv1d (Conv1D)             (None, 100, 6)            1302      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 50, 6)            0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 50, 16)            304       
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 25, 16)           0         
 1D)                                                             
                                